In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as st
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score

In [2]:
dtypes = {'MachineIdentifier':                                    'category',
        'ProductName':                                          'category',
        'EngineVersion':                                        'category',
        'AppVersion':                                           'category',
        'AvSigVersion':                                         'category',
        'IsBeta':                                               'int8',
        'RtpStateBitfield':                                     'float32',
        'IsSxsPassiveMode':                                     'int8',
        'DefaultBrowsersIdentifier':                            'float32',
        'AVProductStatesIdentifier':                            'float32',
        'AVProductsInstalled':                                  'float32',
        'AVProductsEnabled':                                    'float32',
        'HasTpm':                                               'int8',
        'CountryIdentifier':                                    'int16',
        'CityIdentifier':                                       'float32',
        'OrganizationIdentifier':                               'float32',
        'GeoNameIdentifier':                                    'float32',
        'LocaleEnglishNameIdentifier':                          'int8',
        'Platform':                                             'category',
        'Processor':                                            'category',
        'OsVer':                                                'category',
        'OsBuild':                                              'int16',
        'OsSuite':                                              'int16',
        'OsPlatformSubRelease':                                 'category',
        'OsBuildLab':                                           'category',
        'SkuEdition':                                           'category',
        'IsProtected':                                          'float32',
        'AutoSampleOptIn':                                      'int8',
        'PuaMode':                                              'category',
        'SMode':                                                'float32',
        'IeVerIdentifier':                                      'float32',
        'SmartScreen':                                          'category',
        'Firewall':                                             'float32',
        'UacLuaenable':                                         'float32',
        'Census_MDC2FormFactor':                                'category',
        'Census_DeviceFamily':                                  'category',
        'Census_OEMNameIdentifier':                             'float32',
        'Census_OEMModelIdentifier':                            'float32',
        'Census_ProcessorCoreCount':                            'float32',
        'Census_ProcessorManufacturerIdentifier':               'float32',
        'Census_ProcessorModelIdentifier':                      'float32',
        'Census_ProcessorClass':                                'category',
        'Census_PrimaryDiskTotalCapacity':                      'float32',
        'Census_PrimaryDiskTypeName':                           'category',
        'Census_SystemVolumeTotalCapacity':                     'float32',
        'Census_HasOpticalDiskDrive':                           'int8',
        'Census_TotalPhysicalRAM':                              'float32',
        'Census_ChassisTypeName':                               'category',
        'Census_InternalPrimaryDiagonalDisplaySizeInInches':    'float32',
        'Census_InternalPrimaryDisplayResolutionHorizontal':    'float32',
        'Census_InternalPrimaryDisplayResolutionVertical':      'float32',
        'Census_PowerPlatformRoleName':                         'category',
        'Census_InternalBatteryType':                           'category',
        'Census_InternalBatteryNumberOfCharges':                'float32',
        'Census_OSVersion':                                     'category',
        'Census_OSArchitecture':                                'category',
        'Census_OSBranch':                                      'category',
        'Census_OSBuildNumber':                                 'int16',
        'Census_OSBuildRevision':                               'int32',
        'Census_OSEdition':                                     'category',
        'Census_OSSkuName':                                     'category',
        'Census_OSInstallTypeName':                             'category',
        'Census_OSInstallLanguageIdentifier':                   'float32',
        'Census_OSUILocaleIdentifier':                          'int16',
        'Census_OSWUAutoUpdateOptionsName':                     'category',
        'Census_IsPortableOperatingSystem':                     'int8',
        'Census_GenuineStateName':                              'category',
        'Census_ActivationChannel':                             'category',
        'Census_IsFlightingInternal':                           'float32',
        'Census_IsFlightsDisabled':                             'float32',
        'Census_FlightRing':                                    'category',
        'Census_ThresholdOptIn':                                'float32',
        'Census_FirmwareManufacturerIdentifier':                'float32',
        'Census_FirmwareVersionIdentifier':                     'float32',
        'Census_IsSecureBootEnabled':                           'int8',
        'Census_IsWIMBootEnabled':                              'float32',
        'Census_IsVirtualDevice':                               'float32',
        'Census_IsTouchEnabled':                                'int8',
        'Census_IsPenCapable':                                  'int8',
        'Census_IsAlwaysOnAlwaysConnectedCapable':              'float32',
        'Wdft_IsGamer':                                         'float32',
        'Wdft_RegionIdentifier':                                'float32',
        'HasDetections':                                        'int8'}

In [2]:
df = pd.read_csv('train.csv', low_memory=True)
#df = pd.read_csv('train.csv',dtype=dtypes, low_memory=True)

C:\Users\mathg\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (28) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
df.shape

(8921483, 83)

In [4]:
columncounts = df.describe(include='all').loc[['count']]/df.shape[0]
missingcolumns = columncounts.columns[(columncounts < .75).any(axis=0)]
missingcolumns

Index(['DefaultBrowsersIdentifier', 'OrganizationIdentifier', 'PuaMode',
       'SmartScreen', 'Census_ProcessorClass', 'Census_InternalBatteryType',
       'Census_IsFlightingInternal', 'Census_ThresholdOptIn',
       'Census_IsWIMBootEnabled'],
      dtype='object')

In [5]:
df_reduce = df.drop(columns=missingcolumns)
num_reduce = df_reduce.shape[0]

In [6]:
df_reduce.shape

(8921483, 74)

In [9]:
df_nonan = df_reduce.dropna()
df_nonan.shape

(7180589, 74)

In [10]:
df_nonan.shape[0]/df.shape[0]

0.8048649535060483

In [11]:
df = df_nonan

In [22]:
df.describe()

,IsBeta,RtpStateBitfield,IsSxsPassiveMode,AVProductStatesIdentifier,AVProductsInstalled,AVProductsEnabled,HasTpm,CountryIdentifier,CityIdentifier,GeoNameIdentifier,...,Census_FirmwareManufacturerIdentifier,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier,HasDetections
count,7.180589e+06,7.180589e+06,7.180589e+06,7.180589e+06,7.180589e+06,7.180589e+06,7.180589e+06,7.180589e+06,7.180589e+06,7.180589e+06,...,7.180589e+06,7.180589e+06,7.180589e+06,7.180589e+06,7.180589e+06,7.180589e+06,7.180589e+06,7.180589e+06,7.180589e+06,7.180589e+06
mean,1.253379e-06,7.483140e+00,1.816244e-02,4.836533e+04,1.329178e+00,1.018021e+00,9.985071e-01,1.083627e+02,8.102791e+04,1.686621e+02,...,3.991498e+02,3.266665e+04,5.021996e-01,3.037355e-03,1.312086e-01,4.048860e-02,5.842571e-02,2.976463e-01,7.990332e+00,5.069196e-01
std,1.119544e-03,1.156778e+00,1.335386e-01,1.343798e+04,5.343274e-01,1.532739e-01,3.860939e-02,6.314137e+01,4.874690e+04,8.897841e+01,...,2.214988e+02,2.113133e+04,4.999952e-01,5.495070e-02,3.376284e-01,1.971022e-01,2.285078e-01,4.602739e-01,4.561894e+00,4.999522e-01
min,0.000000e+00,0.000000e+00,0.000000e+00,3.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,5.000000e+00,1.000000e+00,...,2.000000e+00,1.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00
25%,0.000000e+00,7.000000e+00,0.000000e+00,4.948000e+04,1.000000e+00,1.000000e+00,1.000000e+00,5.100000e+01,3.682500e+04,8.900000e+01,...,1.420000e+02,1.299300e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.000000e+00,0.000000e+00
50%,0.000000e+00,7.000000e+00,0.000000e+00,5.344700e+04,1.000000e+00,1.000000e+00,1.000000e+00,9.700000e+01,8.237300e+04,1.810000e+02,...,5.000000e+02,3.306600e+04,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+01,1.000000e+00
75%,0.000000e+00,7.000000e+00,0.000000e+00,5.344700e+04,2.000000e+00,1.000000e+00,1.000000e+00,1.630000e+02,1.237000e+05,2.670000e+02,...,5.560000e+02,5.240200e+04,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.100000e+01,1.000000e+00
max,1.000000e+00,3.500000e+01,1.000000e+00,7.049800e+04,6.000000e+00,5.000000e+00,1.000000e+00,2.220000e+02,1.679620e+05,2.960000e+02,...,1.092000e+03,7.210500e+04,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.500000e+01,1.000000e+00


In [17]:
df.describe(include='category')

,MachineIdentifier,ProductName,EngineVersion,AppVersion,AvSigVersion,Platform,Processor,OsVer,OsPlatformSubRelease,OsBuildLab,...,Census_OSVersion,Census_OSArchitecture,Census_OSBranch,Census_OSEdition,Census_OSSkuName,Census_OSInstallTypeName,Census_OSWUAutoUpdateOptionsName,Census_GenuineStateName,Census_ActivationChannel,Census_FlightRing
count,7180589,7180589,7180589,7180589,7180589,7180589,7180589,7180589,7180589,7180589,...,7180589,7180589,7180589,7180589,7180589,7180589,7180589,7180589,7180589,7180589
unique,7180589,4,60,104,7346,3,3,44,9,537,...,415,3,24,28,25,9,5,4,5,9
top,0000028988387b115f69f31a3bf04f09,win8defender,1.1.15200.1,4.18.1807.18075,1.273.1420.0,windows10,x64,10.0.0.0,rs4,17134.1.amd64fre.rs4_release.180410-1804,...,10.0.17134.228,amd64,rs4_release,Core,CORE,UUPUpgrade,FullAuto,IS_GENUINE,Retail,Retail
freq,1,7175886,3265106,4531382,90348,7084153,6561772,7083708,3316622,3108985,...,1235524,6561950,3392667,2828325,2828297,2344218,3249661,6484149,3718751,6759492


In [16]:
df_y = df['HasDetections'].copy()
df_y.value_counts(normalize=True)

1    0.50692
0    0.49308
Name: HasDetections, dtype: float64

In [17]:
df=df.drop(columns='HasDetections')
df=df.drop(columns='MachineIdentifier')
df.columns

Index(['ProductName', 'EngineVersion', 'AppVersion', 'AvSigVersion', 'IsBeta',
       'RtpStateBitfield', 'IsSxsPassiveMode', 'AVProductStatesIdentifier',
       'AVProductsInstalled', 'AVProductsEnabled', 'HasTpm',
       'CountryIdentifier', 'CityIdentifier', 'GeoNameIdentifier',
       'LocaleEnglishNameIdentifier', 'Platform', 'Processor', 'OsVer',
       'OsBuild', 'OsSuite', 'OsPlatformSubRelease', 'OsBuildLab',
       'SkuEdition', 'IsProtected', 'AutoSampleOptIn', 'SMode',
       'IeVerIdentifier', 'Firewall', 'UacLuaenable', 'Census_MDC2FormFactor',
       'Census_DeviceFamily', 'Census_OEMNameIdentifier',
       'Census_OEMModelIdentifier', 'Census_ProcessorCoreCount',
       'Census_ProcessorManufacturerIdentifier',
       'Census_ProcessorModelIdentifier', 'Census_PrimaryDiskTotalCapacity',
       'Census_PrimaryDiskTypeName', 'Census_SystemVolumeTotalCapacity',
       'Census_HasOpticalDiskDrive', 'Census_TotalPhysicalRAM',
       'Census_ChassisTypeName',
       'Census_I

In [98]:
columns = df.columns
for col in columns :
    print(df[col].value_counts(normalize=True))

788e03c511573d3abbd24580af054fdb    1
d5df331f0c2fee335a8df61bc920f46c    1
36712af7639b7292bed2034014f1ab03    1
fe77ba5559257184f7e655db85053245    1
f10a98625c98af528cf0c282c72b002d    1
                                   ..
512d327d292487e5324e4131bd996360    1
e0bff524666cc014810290ba073a1b0f    1
564d9f8b91a926165ff3f33510b704ec    1
50815f9ed701885493c4b6e3cb5815c6    1
8118afc86435d37bb6374bfd3fb9d107    1
Name: MachineIdentifier, Length: 8921483, dtype: int64
win8defender     8826520
mse                94873
mseprerelease         53
scep                  22
windowsintune          8
fep                    7
Name: ProductName, dtype: int64
1.1.15200.1    3845067
1.1.15100.1    3675915
1.1.15000.2     265218
1.1.14901.4     212408
1.1.14600.4     160585
                ...   
1.1.13503.0          2
1.1.11202.0          1
1.1.10701.0          1
1.1.12802.0          1
1.1.11104.0          1
Name: EngineVersion, Length: 70, dtype: int64
4.18.1807.18075    5139224
4.18.1806.18062    

RequireAdmin    4316183
ExistsNotSet    1046183
Off              186553
Warn             135483
Prompt            34533
Block             22533
off                1350
On                  731
&#x02;              416
&#x01;              335
on                  147
requireadmin         10
OFF                   4
0                     3
Promt                 2
Enabled               1
warn                  1
&#x03;                1
00000000              1
requireAdmin          1
prompt                1
Name: SmartScreen, dtype: int64
1.0    8641014
0.0     189119
Name: Firewall, dtype: int64
1.0           8856517
0.0             53851
48.0              206
2.0                30
49.0               17
6357062.0          13
3.0                 6
5.0                 2
16777216.0          1
255.0               1
7798884.0           1
Name: UacLuaenable, dtype: int64
Notebook        5723319
Desktop         1951086
Convertible      405378
Detachable       298233
AllInOne         292077
PCOther   

Core                           3469991
Professional                   3130566
CoreSingleLanguage             1945461
CoreCountrySpecific             166100
ProfessionalEducation            56698
Education                        40704
Enterprise                       35603
ProfessionalN                    28341
EnterpriseS                      20020
ServerStandard                   10128
Cloud                             6275
CoreN                             4790
ServerStandardEval                2751
EducationN                         932
EnterpriseSN                       878
ServerDatacenterEval               829
ServerSolution                     683
EnterpriseN                        351
ProfessionalEducationN             192
ProfessionalWorkstation            128
ServerDatacenter                    15
ProfessionalWorkstationN            13
CloudN                               8
ProfessionalCountrySpecific          5
Ultimate                             4
ServerRdsh               